### Classifier du texte avec un RNN

* Base : https://www.tensorflow.org/text/tutorials/text_classification_rnn 

In [1]:
import tensorflow as tf

### Définition de l'ensemble de données

In [4]:
batch_size = 32
seed = 1

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


2022-07-15 12:43:53.003074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    '/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/aclImdb/test',
    batch_size=batch_size,
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


### Segmentation des ensembles de données

In [6]:
# Pré-traitement de l'ensemble de données 
vocab_size = 1000
output_sequence_length = 150

tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=output_sequence_length,
)

tokenizer.adapt(raw_train_ds.map(lambda text, label: text))

La méthode .adapt définit le vocabulaire de la couche. On les appelle "jetons". Ceux placés en premiers sont : l'espace (' ') et les masques inconnus, ils sont ensuite triés par fréquence.

Voici les 15 premiers :

In [7]:
vocab = tokenizer.get_vocabulary() # list
vocab[:12]

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this']

In [8]:
# Fonction de segmentation de l'ensemble des données
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    vectorized_text = tokenizer(text)
    return vectorized_text, label

In [9]:
# On segmente les ensembles de données
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Et on les prépare à l'entraînement
### Sans Autotune c'est environ 2 sec de plus d'entraînement sur la 
### première epoch et uen sec de plus sur toutes les autres, même
### sur une base de données aussi petite et un modèle aussi simple.

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Avec une dimension faible de l'espace d'encodage, le processus n'est pas complètement réversible. Deux raisons principales à cela :

- La valeur par défaut pour l'arg de TextVectorization(standardize=arg)  est "lower_and_strip_punctuation", c'est à dire que la ponctuatio est retirée et les majuscules replacées par des minuscules.
- La taille limitée du vocabulaire et l'absence de solution de secours basée sur les caractères donnent lieu à des jetons inconnus. (Voir page *Layers/TextVectorization*)

### Création du modèle 

In [10]:
embedding_dim = 20

emb_layer = tf.keras.layers.Embedding(
    input_dim=len(vocab),
    output_dim=embedding_dim,
)

In [14]:
model = tf.keras.Sequential([
    emb_layer,
    tf.keras.layers.SimpleRNN(60),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 20)          20000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 60)                4860      
                                                                 
 dense_2 (Dense)             (None, 40)                2440      
                                                                 
 dense_3 (Dense)             (None, 1)                 41        
                                                                 
Total params: 27,341
Trainable params: 27,341
Non-trainable params: 0
_________________________________________________________________


La couche de TextVect correspond à l'encoder  
La couche d'embedding est ajoutée pour sa capacité à indexer les textes vectorisés d'une manière qui améliore grandement l'apprentissage. Elle bien plus efficace que l'opération consistant à faire passer l'équivalent d'un vecteur codé à chaud à travers une couche Dense.  
La couche de RNN est celle du LSTM  
On termine avec des Dense pour la classification

In [16]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(5e-4),
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
)

In [17]:
epochs = 5
history = model.fit(
    train_ds, 
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/5
625/625 [==============================] - 41s 61ms/step - loss: 0.5332 - binary_accuracy: 0.7149 - val_loss: 0.4765 - val_binary_accuracy: 0.7932
Epoch 2/5
625/625 [==============================] - 40s 64ms/step - loss: 0.4044 - binary_accuracy: 0.8231 - val_loss: 0.4397 - val_binary_accuracy: 0.8096
Epoch 3/5
625/625 [==============================] - 42s 67ms/step - loss: 0.3898 - binary_accuracy: 0.8273 - val_loss: 0.4330 - val_binary_accuracy: 0.8130
Epoch 4/5
625/625 [==============================] - 41s 65ms/step - loss: 0.3818 - binary_accuracy: 0.8326 - val_loss: 0.4242 - val_binary_accuracy: 0.8156
Epoch 5/5
625/625 [==============================] - 42s 67ms/step - loss: 0.3814 - binary_accuracy: 0.8328 - val_loss: 0.4168 - val_binary_accuracy: 0.8148


##### Remarque :  
Il est bien plus rapide de vectoriser les ensembles de données avec avec TextVect puis d'entraîner un modèle sans TextVect sur les bases de données obtenus que d'inclure la couche TextVect dans le Sequential.

In [18]:
loss, accuracy = model.evaluate(test_ds)

print(f"Perte : {loss}, Précision : {accuracy}")

782/782 [==============================] - 15s 20ms/step - loss: 0.4093 - binary_accuracy: 0.8208
Perte : 0.40926551818847656, Précision : 0.8208000063896179


Un peu de dessous de la simple couche d'embedding avec du GlobalPolling..

### Exportation du modèle :

In [19]:
export_model = tf.keras.Sequential([
    tokenizer,
    model,
    tf.keras.layers.Activation('sigmoid')
])

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["accuracy"]
)

In [20]:
export_model.evaluate(raw_test_ds)

782/782 [==============================] - 21s 25ms/step - loss: 0.4093 - accuracy: 0.8208


[0.40926527976989746, 0.8208000063896179]

Les performances sont identiques : ok !

### Prédicitions :

In [26]:
sample_text = ('The movie was not good, it was incredibly good.')

predictions = export_model.predict([sample_text])
print(predictions)

1/1 [==============================] - 0s 74ms/step
[[0.41065142]]
